In [8]:
import urllib2
response = urllib2.urlopen("http://www.pythonscraping.com/pages/page3.html")
html = response.read()
#print(html)

In [10]:
from bs4 import BeautifulSoup

bsObj = BeautifulSoup(html)

for sibling in bsObj.find("table",{"id":"giftList"}).tr.next_siblings:
    print(sibling) 



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

In [13]:
import re

images = bsObj.findAll("img", {"src":re.compile("\.\.\/img\/gifts/img.*\.jpg")})
for image in images: 
    print(image["src"])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


### WikiPedia by Random

In [4]:
import urllib2
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    res = urllib2.urlopen("http://de.wikipedia.org"+articleUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$"))

links = getLinks("/wiki/Basel")
while len(links) > 15:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)


/wiki/Basler_Kantonalbank
/wiki/Kantonalbank
/wiki/%C3%96ffentlich-rechtliches_Kreditinstitut
/wiki/Europ%C3%A4ische_Kommission
/wiki/Simone_Veil
/wiki/Jurist
/wiki/Juristenausbildung_in_der_Schweiz


### get Country

In [6]:
import json
import urllib2


def getCountry(ipAddress):
    response = urllib2.urlopen("http://freegeoip.net/json/"+ipAddress).read().decode('utf-8')
    responseJson = json.loads(response)
    return responseJson.get("country_code")

print(getCountry("50.78.253.58"))
print(getCountry("131.152.1.1"))

US
CH


### Wiki History

In [11]:
import urllib2
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    res = urllib2.urlopen("http://de.wikipedia.org"+articleUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", 
                     href=re.compile("^(/wiki/)((?!:).)*$"))

def getHistoryIPs(pageUrl):
    #Format of revision history pages is: 
    #http://en.wikipedia.org/w/index.php?title=Title_in_URL&action=history
    pageUrl = pageUrl.replace("/wiki/", "")
    historyUrl = "http://de.wikipedia.org/w/index.php?title="+pageUrl+"&action=history"
    print("history url is: "+historyUrl)
    res = urllib2.urlopen(historyUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    #finds only the links with class "mw-anonuserlink" which has IP addresses 
    #instead of usernames
    ipAddresses = bsObj.findAll("a", {"class":"mw-anonuserlink"})
    addressList = set()
    for ipAddress in ipAddresses:
        addressList.add(ipAddress.get_text())
    return addressList


def getCountry(ipAddress):
    try:
        response = urllib2.urlopen("http://freegeoip.net/json/"
                           +ipAddress).read().decode('utf-8')
    except HTTPError:
        return None
    responseJson = json.loads(response)
    return responseJson.get("country_code")
    
links = getLinks("/wiki/Basel")


while(len(links) > 7):
    for link in links:
        print("-------------------") 
        historyIPs = getHistoryIPs(link.attrs["href"])
        for historyIP in historyIPs:
            country = getCountry(historyIP)
            if country is not None:
                print(historyIP+" is from "+country)

    newLink = links[random.randint(0, len(links)-1)].attrs["href"]
    links = getLinks(newLink)

-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Basel_(Begriffskl%C3%A4rung)&action=history
79.246.164.59 is from DE
129.132.211.197 is from CH
91.6.246.237 is from DE
88.72.49.214 is from DE
88.72.33.85 is from DE
217.224.254.59 is from DE
85.1.163.19 is from CH
217.230.59.78 is from DE
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Schweiz&action=history
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Kanton_(Schweiz)&action=history
84.74.177.35 is from CH
2a02:908:fd32:9900:651f:1bbd:8e48:668 is from DE
92.106.225.204 is from CH
5.149.18.72 is from CH
149.172.245.126 is from DE
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Kanton_Basel-Stadt&action=history
178.196.243.56 is from CH
194.153.189.154 is from CH
94.198.248.247 is from CH
83.76.228.210 is from CH
92.76.199.49 is from DE
2002:b253:b780:0:3040:7c5c:a696:9e6e is from CH
109.90.123.147 is from DE
-------

NameError: global name 'HTTPError' is not defined